In [301]:
import json
import numpy as np
from collections import defaultdict
from colors import *

directory = "./outputs/objects"

client_config_file = open("consolidated_config.json","r")
client_config_str = client_config_file.read()
config = json.loads(client_config_str)
client_config_file.close()

whitelist_labels = ["vase","cup","sports ball","mouse"]
angle_threshold = 7.5
ticks_per_packet = 5

In [302]:
screen_center_x = config["image_width"] / 2
horizontal_angle_per_pixel = config["horizontal_FOV"] / config["image_width"]
vertical_angle_per_pixel = config["vertical_FOV"] / config["image_height"]

In [303]:
def find_closest_object(dict_of_numbers, number):
    return min(dict_of_numbers.keys(), key=lambda x:abs((dict_of_numbers[x])-number))

def processDetectedObject(obj,angles_to_each_object):
    # Get the bounding box parameters from Vilib
    y1,x1,y2,x2 = obj["bounding_box"]
    # Scale the boundaries from ratio (0.000 to 1.000) to pixel size (0 to 480/640)
    y1,y2 = y1*config["image_height"],y2*config["image_height"] # Adjust to pixel size
    x1,x2 = x1*config["image_width"],x2*config["image_width"] # Adjust to pixel size
    y1,x1,y2,x2 = int(y1),int(x1),int(y2),int(x2)

    x_center = (x1+x2)/2

    # Find out how many degrees off-center the detected object is
    delta_x = x_center - screen_center_x
    delta_theta = delta_x * horizontal_angle_per_pixel

    # Determine which of the known objects this object is closest to, in terms of angle
    closest_object = find_closest_object(angles_to_each_object,delta_theta)
    closest_angle = angles_to_each_object[closest_object]
    angle_difference = abs(delta_theta - closest_angle)
    return closest_object,angle_difference
# Group data into sets of 5..?
def getAnglesToEachObject(object_locations,vehicle_location):
    vehicle_orientation = (vehicle_location["car_angle"] - vehicle_location["camera_angle"]) % 360
    angles_to_each_object = {}
    for obj in object_locations.keys():
        obj_loc = object_locations[obj]
        theta = np.arctan2(obj_loc["y"]-vehicle_location["y"],obj_loc["x"]-vehicle_location["x"]).item()
        angles_to_each_object[obj] = (vehicle_orientation - (theta * 180 / np.pi)) % 360
    return angles_to_each_object



In [304]:
def getNewDD():
    global defaultdict
    this_dd = defaultdict(float)
    return this_dd

In [305]:
def clientProcessData(data):
    global dd
    all_found_objects = {}

    object_locations = data["object_locations"]
    vehicle_locations = data["vehicle_locations"]
    cv_data = data["raw_data"]
    
    for client_name,data_list in cv_data.items():
        my_location = vehicle_locations[client_name]
        angles_to_each_object = getAnglesToEachObject(object_locations,my_location) # Strictly in Degrees. The expected angles from the robot to each of the listed object locations.
        horizontal_angle_per_pixel = config["horizontal_FOV"] / config["image_width"]
        vertical_angle_per_pixel = config["vertical_FOV"] / config["image_height"]
        screen_center_x = config["image_width"] / 2
        
        found_objects_list = []

        for iteration_count,frame_data_list in enumerate(data_list):
            found_objects = {k: getNewDD() for k in object_locations.keys()}
            for obj in frame_data_list:
                if not "class_name" in obj: continue
                #if not obj["class_name"] in whitelist_labels: continue
                closest_object,angle_difference = processDetectedObject(obj,angles_to_each_object)

                if angle_difference < angle_threshold:
                    this_dd = found_objects[closest_object]
                    this_dd[obj["class_name"]] += obj["score"]

            if iteration_count%ticks_per_packet == ticks_per_packet-1:
                # Process the data
                for this_dd in found_objects.values():
                    for key in this_dd.keys():
                        this_dd[key] /= ticks_per_packet
                # Send the data
                found_objects_list.append(found_objects)
        # At the end of each client's loop, before proceeding...
        all_found_objects[client_name] = found_objects_list
    # After all clients have finished processing...
    return all_found_objects

In [306]:
class Client:
    def __init__(self,name):
        self.name = name
        self.data = []
        self.reliability = 0.5
        self.location = {"x":0,"y":0,"theta":0}
        self.record=[0,0]
    def setData(self,data):
        self.data = data
    def getData(self):
        return self.data
    def getName(self):
        return self.name
    def getReliability(self):
        return self.reliability
    def changeReliability(self,reliability):
        new_reliability = self.reliability + reliability
        if new_reliability < 0.3: new_reliability = 0.3
        elif new_reliability > 1.0: new_reliability = 1.0
        self.reliability = new_reliability
    def getLocation(self):
        return self.location
    def setLocation(self,x,y,theta):
        self.location = {"x":x,"y":y,"theta":theta}
    def addWin(self):
        self.record[0] += 1
    def addLoss(self):
        self.record[1] += 1
    def getRecord(self):
        return self.record
    def getWinRatio(self):
        if self.record[1] == 0: return 1
        return self.record[0]/(self.record[1]+self.record[0])

client_list = {}

In [307]:
def compare_verdicts(verdict1,verdict2):
    return len([i for i in verdict1.keys() if verdict1[i] == verdict2[i]]) / 2 - 1

def normalize_angle(angle):
    return angle / 360.0

def normalize_distance(distance, max_distance=100):
    return 1 - (distance / max_distance)

def calculate_visibility_score(angle, distance):
    return 0.3 * normalize_angle(angle) + 0.7 * normalize_distance(distance)

def get_total_weight(reliability,confidence,angle,distance):
    visibility_score = calculate_visibility_score(angle,distance)
    return reliability * confidence * visibility_score

def getAngleFromClientToObject(client,obj_pos):
    my_pos = client.getLocation()
    theta = np.arctan2(obj_pos["y"]-my_pos["y"],obj_pos["x"]-my_pos["x"]).item()
    return (my_pos["theta"] - (theta * 180 / np.pi)) % 360

def get_overall_score(client,confidence,obj_pos):
    my_pos = client.getLocation()
    angle = getAngleFromClientToObject(client,obj_pos)
    distance = np.sqrt((obj_pos["x"]-my_pos["x"])**2 + (obj_pos["y"]-my_pos["y"])**2)
    reliability = client.getReliability()
    return get_total_weight(reliability,confidence,angle,distance)

In [308]:
def getLocalVerdict(client,data,object_locations):
    verdict = {x:None for x in data.keys()}
    for obj_id,obj_data in data.items():
        obj_loc = object_locations[obj_id]
        if len(obj_data) == 0:
            verdict[obj_id] = None
            client.addLoss()
            continue
        chosen = max(obj_data.keys(), key=lambda x:obj_data[x])
        if chosen in obj_loc["identities"]:
            client.addWin()
        else:
            client.addLoss()
        verdict[obj_id] = chosen
    return verdict

def serverProcessData(data,original_data):
    global dd
    global client_list
    object_locations = original_data["object_locations"]
    vehicle_locations = original_data["vehicle_locations"]
    verdict_list = []

    win_count = 0
    loss_count = 0

    for client_name in config["vehicle_locations"].keys():
        client_list[client_name] = Client(client_name)

    for iteration in range(200):
        verdict = {x:None for x in object_locations.keys()}
        dd_dict = {x: getNewDD() for x in object_locations.keys()}
        for client_name,client in client_list.items():
            my_loc = vehicle_locations[client_name]
            client.setLocation(my_loc["x"],my_loc["x"],my_loc["car_angle"]-my_loc["camera_angle"])
            client_data = data[client_name]
            frame_data = client_data[iteration]

            for obj_id,obj_data in frame_data.items():
                for obj_class,obj_confidence in obj_data.items():
                    if obj_confidence == 0: continue
                    obj_pos = object_locations[obj_id]
                    score = get_overall_score(client,obj_confidence,obj_pos)
                    dd_dict[obj_id][obj_class] += score
        
        for obj_id,dd in dd_dict.items():
            if len(dd) == 0:
                verdict[obj_id] = None
                loss_count += 1
                continue
            decision = max(dd.keys(), key=lambda x:dd[x])
            verdict[obj_id] = decision
            if decision in object_locations[obj_id]["identities"]:
                win_count += 1
            else:
                loss_count += 1

        for client_name,client in client_list.items():
            client_data = data[client_name][iteration]
            local_verdict = getLocalVerdict(client,client_data,object_locations)
            result = compare_verdicts(verdict,local_verdict)
            client.changeReliability(result * 0.01)
        
        verdict_list.append(verdict)
    
    accuracies = [x.getWinRatio() for x in client_list.values()]
    accuracies.insert(0,win_count/(win_count+loss_count))
    return accuracies,verdict_list

In [309]:
for i in range(1,10):
    with open(f"{directory}/output_{i}.json") as f:
        data = json.load(f)
        f.close()
        #print("Loaded data from file",i)
        all_found_objects = clientProcessData(data)
        #print("Finished client-side processing")
        accuracies,verdict_list = serverProcessData(all_found_objects,data)
        server,a,b,c,d = accuracies

        server_acc = np.round(server*100,3)
        avg_acc = np.round((a+b+c+d)/4*100,3)
        diff = np.round(server_acc-avg_acc,3)

        print(f"Server accuracy:          {getGreen(server_acc)}%")
        print(f"Avg. client accuracy:     {getCyan(avg_acc)}%")
        print(f"Difference:              +{getRed(diff)}%")
        print()
        #print(f"{a:.2f},{b:.2f},{c:.2f},{d:.2f}")
        #print("Finished server-side processing")
        #if i == 1:
            #print(verdict_list)

print("All done! :)")

Server accuracy:          99.667%
Avg. client accuracy:     38.042%
Difference:              +61.625%

Server accuracy:          99.5%
Avg. client accuracy:     38.583%
Difference:              +60.917%

Server accuracy:          99.5%
Avg. client accuracy:     38.875%
Difference:              +60.625%

Server accuracy:          61.333%
Avg. client accuracy:     15.333%
Difference:              +46.0%

Server accuracy:          63.5%
Avg. client accuracy:     15.875%
Difference:              +47.625%

Server accuracy:          64.333%
Avg. client accuracy:     16.083%
Difference:              +48.25%

Server accuracy:          99.0%
Avg. client accuracy:     24.792%
Difference:              +74.208%

Server accuracy:          99.667%
Avg. client accuracy:     24.917%
Difference:              +74.75%

Server accuracy:          99.167%
Avg. client accuracy:     24.833%
Difference:              +74.334%

All done! :)
